## Installation

In [ ]:
%pip install yacs
%pip install termcolor
%pip install opencv-python

  Using cached yacs-0.1.8-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


Resize Image

In [5]:

import numpy as np
import cv2 as cv
from PIL import Image
# from google.colab.patches import cv2_imshow 

import torchvision.transforms as transforms
import sys
sys.path.append("../packnet-sfm/")
from packnet_sfm.datasets.augmentations import resize_image


import os 
DATA_PATH = "../Kitti/3/"
test = os.listdir(DATA_PATH)

for i in test:
  img = cv.imread(DATA_PATH+i)
  img = resize_image(Image.fromarray(cv.cvtColor(img, cv.COLOR_BGR2RGB)), (384,1280))
  cv.imwrite(DATA_PATH+i, cv.cvtColor(np.array(img),cv.COLOR_BGR2RGB))

## Output Depth

In [1]:
import argparse
import numpy as np
import os
import torch

from glob import glob
from cv2 import imwrite
# from google.colab.patches import cv2_imshow 

import sys
sys.path.append("../packnet-sfm/")

from packnet_sfm.models.model_wrapper import ModelWrapper
from packnet_sfm.datasets.augmentations import resize_image, to_tensor
from packnet_sfm.utils.horovod import hvd_init, rank, world_size, print0
from packnet_sfm.utils.image import load_image
from packnet_sfm.utils.config import parse_test_file
from packnet_sfm.utils.load import set_debug
from packnet_sfm.utils.depth import write_depth, inv2depth, viz_inv_depth
from packnet_sfm.utils.logging import pcolor

def is_image(file, ext=('.png', '.jpg',)):
    """Check if a file is an image with certain extensions"""
    return file.endswith(ext)


def parse_args():
    parser = argparse.ArgumentParser(description='PackNet-SfM inference of depth maps from images')
    parser.add_argument('--checkpoint', type=str, help='Checkpoint (.ckpt)')
    parser.add_argument('--input', type=str, help='Input file or folder')
    parser.add_argument('--output', type=str, help='Output file or folder')
    parser.add_argument('--image_shape', type=int, nargs='+', default=None,
                        help='Input and output image shape '
                             '(default: checkpoint\'s config.datasets.augmentation.image_shape)')
    parser.add_argument('--half', action="store_true", help='Use half precision (fp16)')
    parser.add_argument('--save', type=str, choices=['npz', 'png'], default=None,
                        help='Save format (npz or png). Default is None (no depth map is saved).')
    args = parser.parse_args()
    assert args.checkpoint.endswith('.ckpt'), \
        'You need to provide a .ckpt file as checkpoint'
    assert args.image_shape is None or len(args.image_shape) == 2, \
        'You need to provide a 2-dimensional tuple as shape (H,W)'
    assert (is_image(args.input) and is_image(args.output)) or \
           (not is_image(args.input) and not is_image(args.input)), \
        'Input and output must both be images or folders'
    return args


@torch.no_grad()
def infer_and_save_depth(input_file, output_file, model_wrapper, image_shape, half, save):
    """
    Process a single input file to produce and save visualization
    Parameters
    ----------
    input_file : str
        Image file
    output_file : str
        Output file, or folder where the output will be saved
    model_wrapper : nn.Module
        Model wrapper used for inference
    image_shape : Image shape
        Input image shape
    half: bool
        use half precision (fp16)
    save: str
        Save format (npz or png)
    """
    if not is_image(output_file):
        # If not an image, assume it's a folder and append the input name
        os.makedirs(output_file, exist_ok=True)
        output_file = os.path.join(output_file, os.path.basename(input_file))

    # change to half precision for evaluation if requested
    dtype = torch.float16 if half else None

    # Load image
    image = load_image(input_file)
    # Resize and to tensor
    image = resize_image(image, image_shape)
    image = to_tensor(image).unsqueeze(0)

    # Send image to GPU if available
    if torch.cuda.is_available():
        image = image.to('cuda:{}'.format(rank()), dtype=dtype)
        
    # Depth inference (returns predicted inverse depth)
    pred_inv_depth = model_wrapper.depth(image)['inv_depths'][0]
    
    if save == 'npz' or save == 'png':
        # Get depth from predicted depth map and save to different formats
        filename = '{}.{}'.format(os.path.splitext(output_file)[0], save)
        print('Saving {} to {}'.format(
            pcolor(input_file, 'cyan', attrs=['bold']),
            pcolor(filename, 'magenta', attrs=['bold'])))
        write_depth(filename, depth=inv2depth(pred_inv_depth))
        # print(inv2depth(pred_inv_depth))
    else:
        # Prepare RGB image
        rgb = image[0].permute(1, 2, 0).detach().cpu().numpy() * 255
        # Prepare inverse depth
        viz_pred_inv_depth = viz_inv_depth(pred_inv_depth[0]) * 255
        # Concatenate both vertically
        # image = np.concatenate([rgb, viz_pred_inv_depth], 0)
        image = viz_pred_inv_depth
        # Save visualization
        sys.stdout.write('Saving {} to {}'.format(
            pcolor(input_file, 'cyan', attrs=['bold']),
            pcolor(output_file, 'magenta', attrs=['bold'])))
        sys.stdout.flush()
        imwrite(output_file, image[:, :, ::-1])
        # cv2_imshow(image[:,:,::-1])


def main(args):

    # Initialize horovod
    hvd_init()

    # Parse arguments
    config, state_dict = parse_test_file(args['ckpt'])

    # If no image shape is provided, use the checkpoint one
    image_shape = ""
    if "imgshp" in args:
        image_shape=args['imgshp']
    else:
        image_shape = config.datasets.augmentation.image_shape

    # Set debug if requested
    set_debug(config.debug)

    # Initialize model wrapper from checkpoint arguments
    model_wrapper = ModelWrapper(config, load_datasets=False)
    # Restore monodepth_model state
    model_wrapper.load_state_dict(state_dict)

    # change to half precision for evaluation if requested
    dtype = torch.float16 if "half" in args else None

    # Send model to GPU if available
    if torch.cuda.is_available():
        model_wrapper = model_wrapper.to('cuda:{}'.format(rank()), dtype=dtype)
    # Set to eval mode
    model_wrapper.eval()

    if os.path.isdir(args["input"]):
        # If input file is a folder, search for image files
        files = []
        for ext in ['png', 'jpg']:
            files.extend(glob((os.path.join(args["input"], '*.{}'.format(ext)))))
        files.sort()
        print0('Found {} files'.format(len(files)))
    else:
        # Otherwise, use it as is
        files = [args["input"]]

    # Process each file
    for fn in files[rank()::world_size()]:
        infer_and_save_depth(
            fn, args["output"], model_wrapper, image_shape, args["half"], args["save"])



if __name__ == '__main__':
    # args = parse_args()
    datapath = "../Kitti/"
    args = {}
    args['ckpt'] = "../packnet-sfm/PackNet01_HR_velsup_CStoK.ckpt"
    args['input'] = datapath+"3"
    args['output'] = datapath+"3_depth"
    args['save'] = "npz"
    args['half'] = True
    args['imgshp'] = (384,1280)
    main(args)

### Preparing Model
Model: SelfSupModel
DepthNet: PackNet01
PoseNet: PoseNet
Found 158 files
Saving ../Kitti/3\0000000000.png to ../Kitti/3_depth\0000000000.npz
Saving ../Kitti/3\0000000001.png to ../Kitti/3_depth\0000000001.npz
Saving ../Kitti/3\0000000002.png to ../Kitti/3_depth\0000000002.npz
Saving ../Kitti/3\0000000003.png to ../Kitti/3_depth\0000000003.npz
Saving ../Kitti/3\0000000004.png to ../Kitti/3_depth\0000000004.npz
Saving ../Kitti/3\0000000005.png to ../Kitti/3_depth\0000000005.npz
Saving ../Kitti/3\0000000006.png to ../Kitti/3_depth\0000000006.npz
Saving ../Kitti/3\0000000007.png to ../Kitti/3_depth\0000000007.npz
Saving ../Kitti/3\0000000008.png to ../Kitti/3_depth\0000000008.npz
Saving ../Kitti/3\0000000009.png to ../Kitti/3_depth\0000000009.npz
Saving ../Kitti/3\0000000010.png to ../Kitti/3_depth\0000000010.npz
Saving ../Kitti/3\0000000011.png to ../Kitti/3_depth\0000000011.npz
Saving ../Kitti/3\0000000012.png to ../Kitti/3_depth\0000000012.npz
Saving ../Kitti/3\00000

In [4]:
import torch
torch.cuda.is_available()
torch.__version__

'1.11.0+cpu'